In [6]:
import os
import mlflow
from configparser import ConfigParser
config = ConfigParser()
config.read('./my_auth_config.ini')

os.environ['MLFLOW_TRACKING_USERNAME'] = config['mlflow']['TRACKING_USERNAME']
os.environ['MLFLOW_TRACKING_PASSWORD'] = config['mlflow']['TRACKING_PASSWORD']

mlflow.set_tracking_uri(config['mlflow']['URL'])
mlflow.set_experiment("test-experiment")

mlflow.log_metric("test-metric", 1.0)
mlflow.end_run()

2025/01/17 20:40:25 INFO mlflow.tracking.fluent: Experiment with name 'test-experiment' does not exist. Creating a new experiment.


In [2]:
import mlflow

# Get the current run and its artifact location
run = mlflow.active_run()
print("Artifact URI:", run.info.artifact_uri)

Artifact URI: mlflow-artifacts:/1/69ee47a684544d109c13076f50d5c19c/artifacts


In [8]:
mlflow.end_run()

🏃 View run abundant-auk-490 at: http://mlflow.localhost/#/experiments/1/runs/7c6d251010c540e99331931688e77b8d
🧪 View experiment at: http://mlflow.localhost/#/experiments/1


In [9]:
# Log a run
with mlflow.start_run():
    mlflow.log_param("param1", 5)
    mlflow.log_metric("metric1", 0.75)
    with open("test.txt", "w") as f:
        f.write("This is a test artifact.")
    mlflow.log_artifact("test.txt")

🏃 View run silent-yak-898 at: http://mlflow.localhost/#/experiments/1/runs/723874681f7843b1b9640a3069c9b772
🧪 View experiment at: http://mlflow.localhost/#/experiments/1


In [14]:
mlflow.end_run()

🏃 View run polite-calf-862 at: http://mlflow.localhost/#/experiments/1/runs/5440ba024edf495fb29c9d6e3b96367a
🧪 View experiment at: http://mlflow.localhost/#/experiments/1


In [13]:
import pandas as pd
import numpy as np

np.random.seed(42)  # for reproducibility
df = pd.DataFrame({
    'sales': np.random.normal(1000, 200, 100),
    'customers': np.random.randint(50, 200, 100),
    'satisfaction': np.random.uniform(3.5, 5.0, 100)
})

# prepare features and target
X = df.drop(columns=['sales'])
y = df['sales']

# get model from sklearn
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

# train and log the model with MLflow
with mlflow.start_run(run_name="random_forest_example1"):
    # Create and train the model
    mlflow.sklearn.autolog()
    model = RandomForestRegressor(n_estimators=2, random_state=42)
    model.fit(X, y)

    # Make predictions and calculate metrics
    predictions = model.predict(X)
    mse = mean_squared_error(y, predictions)
    r2 = r2_score(y, predictions)

    # Log parameters
    mlflow.log_param("n_estimators", 100)
    mlflow.log_param("random_state", 42)

    # Log metrics
    mlflow.log_metric("mse", mse)
    mlflow.log_metric("r2", r2)

    # Create an input example
    input_example = X.iloc[[0]]  # Using the first row as an example

    # Log the model with signature and input example
    print("logging model")
    mlflow.sklearn.log_model(
        model,
        "random_forest_model",
        input_example=input_example,  # Add input example
        signature=mlflow.models.infer_signature(X, predictions)  # Add model signature
    )


2025/01/18 00:36:09 WARNING mlflow.utils.autologging_utils: MLflow sklearn autologging is known to be compatible with 0.24.1 <= scikit-learn <= 1.5.2, but the installed version is 1.6.1. If you encounter errors during autologging, try upgrading / downgrading scikit-learn to a compatible version, or try upgrading MLflow.
2025/01/18 00:36:09 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/Users/parsa/Desk/projects/university/slmops-project/infra/provision/zen/.venv/lib/python3.11/site-packages/mlflow/types/utils.py:435: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer

🏃 View run random_forest_example1 at: http://mlflow.localhost/#/experiments/1/runs/f2234f0435424035aeaba61354a169f7
🧪 View experiment at: http://mlflow.localhost/#/experiments/1


MlflowException: INVALID_PARAMETER_VALUE: Changing param values is not allowed. Param with key='n_estimators' was already logged with value='2' for run ID='f2234f0435424035aeaba61354a169f7'. Attempted logging new value '100'.

The cause of this error is typically due to repeated calls
to an individual run_id event logging.

Incorrect Example:
---------------------------------------
with mlflow.start_run():
    mlflow.log_param("depth", 3)
    mlflow.log_param("depth", 5)
---------------------------------------

Which will throw an MlflowException for overwriting a
logged parameter.

Correct Example:
---------------------------------------
with mlflow.start_run():
    with mlflow.start_run(nested=True):
        mlflow.log_param("depth", 3)
    with mlflow.start_run(nested=True):
        mlflow.log_param("depth", 5)
---------------------------------------

Which will create a new nested run for each individual
model and prevent parameter key collisions within the
tracking store.

In [20]:
try:
    mlflow.tracking.MlflowClient().get_experiment_by_name("test-experiment")
    print(f"Successfully connected to MLflow server at {config['mlflow']['URL']}")
except Exception as e:
    print(f"Failed to connect to MLflow server: {e}")
    raise

Successfully connected to MLflow server at http://mlflow.localhost


In [23]:
with mlflow.start_run(run_name="random_forest_minimal"):
    # Create and train a minimal model
    model = RandomForestRegressor(n_estimators=2, random_state=42)  # Using minimal n_estimators
    # model.fit(X, y)

    # Log basic parameters
    mlflow.log_param("n_estimators", 2)
    print("Parameters logged successfully")

    # Step 1: Try basic model logging without signature or input example
    try:
        print("Attempting to log model...")
        mlflow.sklearn.log_model(
            model,
            "random_forest_model"
        )
        print("Basic model logging successful")
    except Exception as e:
        print(f"Failed to log basic model: {e}")
        raise

    # If the above works, uncomment the next section to try with signature
    # try:
    #     predictions = model.predict(X)
    #     mlflow.sklearn.log_model(
    #         model,
    #         "random_forest_model_with_signature",
    #         signature=mlflow.models.infer_signature(X, predictions)
    #     )
    #     print("Model with signature logged successfully")
    # except Exception as e:
    #     print(f"Failed to log model with signature: {e}")

Parameters logged successfully
Attempting to log model...


2025/01/17 17:27:17 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
2025/01/17 17:27:17 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run random_forest_minimal at: http://mlflow.localhost/#/experiments/1/runs/45c4281c14a7456f8c3d72ecd0611857
🧪 View experiment at: http://mlflow.localhost/#/experiments/1


KeyboardInterrupt: 